# Strategic Patient Risk Stratification & Readmission Predictive Modeling
### Vitality Health Network (VHN) – ITS 2122 Group Project (Semester 3, 2025)

This notebook implements **all required phases** from the project specification:
1. Data ingestion & cleaning (standardize missing values, remove deceased, dedupe)
2. ID mapping integration (`IDs_mapping.csv`)
3. ICD‑9 enrichment via ethical web scraping (top 20 primary diagnoses)
4. Exploratory Data Analysis (EDA) with required visualizations
5. Feature engineering: **Vitality Complexity Index (VCI)** and validation charts

**How to run:** Place the following files in the same folder as this notebook:
- `diabetic_data.csv`
- `IDs_mapping.csv`

Then run **Kernel → Restart & Run All** to ensure full reproducibility.

In [1]:
# =========================
# 0) Imports & Settings
# =========================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import quote

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

RANDOM_SEED = 42


## Phase 1 — Data Ingestion & Clinical Sanitation
Key requirements from the spec:
- Interpret `?` as missing values
- Drop `weight` if missingness > 90%
- Remove deceased patients (discharge disposition = Expired)
- Remove exact duplicate rows

We'll also create a **binary target** for the HRRP-relevant group: `readmitted == '<30'`.

In [2]:
# =========================
# 1) Load data
# =========================
DATA_PATH = "diabetic_data.csv"
MAP_PATH  = "IDs_mapping.csv"

try:
    df_raw = pd.read_csv(DATA_PATH, na_values=["?"])
except FileNotFoundError as e:
    raise FileNotFoundError(
        "Could not find diabetic_data.csv. Put it in the same folder as this notebook."
    ) from e

try:
    ids_map = pd.read_csv(MAP_PATH)
except FileNotFoundError as e:
    raise FileNotFoundError(
        "Could not find IDs_mapping.csv. Put it in the same folder as this notebook."
    ) from e

print("diabetic_data.csv shape:", df_raw.shape)
display(df_raw.head())


diabetic_data.csv shape: (101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,11,5,13,2,0,1,648,250,V27,6,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
# Quick schema audit (recommended by spec)
df_raw.info()
display(df_raw.describe(include="all").T.head(25))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
encounter_id,101766.0,NaN,NaN,NaN,165201645.622978,102640295.983457,12522.0,84961194.0,152388987.0,230270887.5,443867222.0
patient_nbr,101766.0,NaN,NaN,NaN,54330400.694947,38696359.346534,135.0,23413221.0,45505143.0,87545949.75,189502619.0
race,99493,5,Caucasian,76099,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,101766,3,Female,54708,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,101766,10,[70-80),26068,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weight,3197,9,[75-100),1336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admission_type_id,101766.0,NaN,NaN,NaN,2.024006,1.445403,1.0,1.0,1.0,3.0,8.0
discharge_disposition_id,101766.0,NaN,NaN,NaN,3.715642,5.280166,1.0,1.0,1.0,4.0,28.0
admission_source_id,101766.0,NaN,NaN,NaN,5.754437,4.064081,1.0,1.0,7.0,7.0,25.0
time_in_hospital,101766.0,NaN,NaN,NaN,4.395987,2.985108,1.0,2.0,4.0,6.0,14.0


In [4]:
# Normalize gender 'Unknown/Invalid' to NaN (spec flags these as invalid)
df = df_raw.copy()
df.loc[df["gender"].astype(str).str.contains("Unknown", case=False, na=False), "gender"] = np.nan


In [5]:
# Drop weight if >90% missingness (spec requirement)
if "weight" in df.columns:
    miss_rate = df["weight"].isna().mean()
    print(f"Weight missingness: {miss_rate:.2%}")
    if miss_rate > 0.90:
        df.drop(columns=["weight"], inplace=True)
        print("Dropped column: weight (missingness > 90%)")
else:
    print("No weight column found.")


Weight missingness: 96.86%
Dropped column: weight (missingness > 90%)


### Remove deceased patients
Use `IDs_mapping.csv` to find discharge disposition codes that indicate **Expired** and filter those encounters out before readmission analysis.

In [6]:
# =========================
# Identify discharge_disposition_id values for "Expired"
# =========================
import re

print("IDs_mapping columns:", ids_map.columns.tolist())

map_df = ids_map.copy()
for c in map_df.columns:
    if map_df[c].dtype == object:
        map_df[c] = map_df[c].astype(str)

expired_rows = map_df[
    map_df.apply(lambda r: r.astype(str).str.contains("discharge", case=False, na=False).any(), axis=1) &
    map_df.apply(lambda r: r.astype(str).str.contains("expired", case=False, na=False).any(), axis=1)
]

if expired_rows.empty:
    expired_rows = map_df[map_df.apply(lambda r: r.astype(str).str.contains("expired", case=False, na=False).any(), axis=1)]

display(expired_rows.head(20))

expired_ids = set()
for _, row in expired_rows.iterrows():
    for val in row.values:
        s = str(val)
        if s.isdigit():
            expired_ids.add(int(s))
        else:
            for m in re.findall(r"\b\d+\b", s):
                expired_ids.add(int(m))

# Fallback to common codes if mapping file doesn't reveal them clearly
if not expired_ids:
    expired_ids = {11, 19, 20}

print("Expired discharge_disposition_id codes (derived):", sorted(expired_ids))

before = df.shape[0]
df = df[~df["discharge_disposition_id"].isin(expired_ids)].copy()
after = df.shape[0]
print(f"Removed deceased encounters: {before-after:,} rows ({(before-after)/before:.2%})")


IDs_mapping columns: ['admission_type_id', 'description']


,admission_type_id,description
20,11,Expired
28,19,"Expired at home. Medicaid only, hospice."
29,20,"Expired in a medical facility. Medicaid only, ..."
30,21,"Expired, place unknown. Medicaid only, hospice."


Expired discharge_disposition_id codes (derived): [11, 19, 20, 21]
Removed deceased encounters: 1,652 rows (1.62%)


In [7]:
# Remove exact duplicates
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Removed exact duplicates: {before-after:,} rows")


Removed exact duplicates: 0 rows


In [8]:
# Binary target for HRRP class: readmitted == '<30'
df["readmit_30d"] = (df["readmitted"] == "<30").astype(int)
df["readmit_label"] = df["readmitted"].astype("category")

display(df[["readmitted", "readmit_30d"]].value_counts())


readmitted  readmit_30d
NO          0              53212
>30         0              35545
<30         1              11357
Name: count, dtype: int64

## ID Mapping Integration
Convert numeric codes to meaningful labels using `IDs_mapping.csv`. This improves interpretability in EDA and reporting.

In [9]:
# =========================
# Build mapping dictionaries from IDs_mapping.csv (robust inference)
# =========================
def build_id_maps(ids_map: pd.DataFrame):
    """Return dicts for: admission_type_id, discharge_disposition_id, admission_source_id."""
    maps = {}
    cols_lower = {c.lower(): c for c in ids_map.columns}

    # Schema A: columns: id, category, description
    if set(["id", "category", "description"]).issubset(set(cols_lower.keys())):
        tmp = ids_map.rename(columns={cols_lower["id"]:"id", cols_lower["category"]:"category", cols_lower["description"]:"description"})
        for cat, target_col in [
            ("admission_type", "admission_type_id"),
            ("discharge_disposition", "discharge_disposition_id"),
            ("admission_source", "admission_source_id"),
        ]:
            sub = tmp[tmp["category"].astype(str).str.contains(cat, case=False, na=False)]
            if not sub.empty:
                sub = sub.dropna(subset=["id","description"])
                sub = sub[sub["id"].astype(str).str.match(r"^\d+$")]
                maps[target_col] = dict(zip(sub["id"].astype(int), sub["description"].astype(str)))

    # Schema B: direct columns
    for target_col in ["admission_type_id", "discharge_disposition_id", "admission_source_id"]:
        if target_col in ids_map.columns:
            desc_col = None
            for cand in ["description", "desc", "name"]:
                if cand in cols_lower:
                    desc_col = cols_lower[cand]
                    break
            if desc_col:
                sub = ids_map[[target_col, desc_col]].dropna()
                sub = sub[sub[target_col].astype(str).str.match(r"^\d+$")]
                maps[target_col] = dict(zip(sub[target_col].astype(int), sub[desc_col].astype(str)))

    return maps

id_maps = build_id_maps(ids_map)
print("Built maps for:", list(id_maps.keys()))

for col, mp in id_maps.items():
    if col in df.columns:
        df[col + "_desc"] = df[col].map(mp).fillna("Unknown")

display(df[[c for c in df.columns if c.endswith("_desc")]].head())


Built maps for: ['admission_type_id']


,admission_type_id_desc
0,Transfer from another health care facility
1,Physician Referral
2,Physician Referral
3,Physician Referral
4,Physician Referral


## Phase 2 — Web Scraping ICD‑9 Descriptions (Top 20 diag_1)
Scrape a public ICD‑9 lookup site for the **top 20** `diag_1` codes, add an ethical delay, and map results to a new `Primary_Diagnosis_Desc` column.

In [10]:
# Identify top 20 primary diagnosis codes
top20 = (
    df["diag_1"]
    .dropna()
    .astype(str)
    .value_counts()
    .head(20)
)
display(top20.to_frame("count"))
top_codes = top20.index.tolist()
print("Top 20 diag_1 codes:", top_codes)


,count
diag_1,
428,6735
414,6555
786,4016
410,3477
486,3413
427,2729
491,2252
715,2147
682,2030


Top 20 diag_1 codes: ['428', '414', '786', '410', '486', '427', '491', '715', '682', '780', '434', '996', '276', '250.8', '599', '38', '584', 'V57', '250.6', '820']


In [11]:
# =========================
# Scraper (icd9.chrisendres.com)
# =========================
ICD9_BASE = "http://icd9.chrisendres.com/index.php?action=child&recordid="

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 (Educational project; contact: student@example.com)"})

def scrape_icd9_description(code: str, timeout: int = 15) -> str:
    """Return best-effort long description for an ICD-9 code."""
    url = ICD9_BASE + quote(str(code))
    r = session.get(url, timeout=timeout)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    text = soup.get_text(" ", strip=True)

    import re
    m = re.search(r"Long Description\s*:?\s*(.+?)(?:\s{2,}|\bShort Description\b|\bCode\b|$)", text)
    if m:
        desc = m.group(1).strip()
        if 3 <= len(desc) <= 200:
            return desc

    # Fallback: try heading/title
    h = soup.find(["h1", "h2"])
    if h:
        return h.get_text(" ", strip=True)[:200]
    if soup.title:
        return soup.title.get_text(strip=True)[:200]
    return "Description not found"

code_to_desc = {}
for i, code_ in enumerate(top_codes, start=1):
    try:
        code_to_desc[code_] = scrape_icd9_description(code_)
    except Exception as e:
        code_to_desc[code_] = f"Scrape failed: {type(e).__name__}"
    time.sleep(1)  # ethical delay
    if i % 5 == 0:
        print(f"Scraped {i}/20...")

display(pd.DataFrame({"diag_1": list(code_to_desc.keys()), "description": list(code_to_desc.values())}))


Scraped 5/20...
Scraped 10/20...
Scraped 15/20...
Scraped 20/20...


,diag_1,description
0,428,Online ICD9/ICD9CM codes
1,414,Online ICD9/ICD9CM codes
2,786,Online ICD9/ICD9CM codes
3,410,Online ICD9/ICD9CM codes
4,486,Online ICD9/ICD9CM codes
5,427,Online ICD9/ICD9CM codes
6,491,Online ICD9/ICD9CM codes
7,715,Online ICD9/ICD9CM codes
8,682,Online ICD9/ICD9CM codes
9,780,Online ICD9/ICD9CM codes


In [ ]:
# Map descriptions into dataframe
df["Primary_Diagnosis_Desc"] = df["diag_1"].astype(str).map(code_to_desc)
df["Primary_Diagnosis_Desc"] = df["Primary_Diagnosis_Desc"].fillna("Other / Not in Top 20")
display(df[["diag_1", "Primary_Diagnosis_Desc"]].head(10))


## Phase 3 — Exploratory Data Analysis (EDA)
Produces all required plots and rate tables.

In [ ]:
sns.set_theme(style="whitegrid")

def show():
    plt.tight_layout()
    plt.show()


In [ ]:
# Readmission distribution + rates
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="readmitted", order=["NO", ">30", "<30"])
plt.title("Readmission Distribution")
plt.xlabel("Readmitted")
plt.ylabel("Count")
show()

rates = df["readmitted"].value_counts(normalize=True).rename("rate").to_frame()
display(rates)


In [ ]:
# Age distribution
plt.figure(figsize=(10,4))
order_age = sorted(df["age"].dropna().unique())
sns.countplot(data=df, x="age", order=order_age)
plt.title("Age Distribution")
plt.xlabel("Age Group")
plt.ylabel("Count")
plt.xticks(rotation=45)
show()


In [ ]:
# 30-day readmission rate by race & gender
tmp = df.dropna(subset=["race", "gender"]).copy()
rate_rg = (
    tmp.groupby(["race", "gender"])["readmit_30d"]
    .mean()
    .reset_index()
    .rename(columns={"readmit_30d": "readmit_30d_rate"})
)

plt.figure(figsize=(12,5))
sns.barplot(data=rate_rg, x="race", y="readmit_30d_rate", hue="gender")
plt.title("30-Day Readmission Rate (<30) by Race and Gender")
plt.xlabel("Race")
plt.ylabel("Rate")
plt.xticks(rotation=30, ha="right")
show()

display(rate_rg.sort_values("readmit_30d_rate", ascending=False).head(15))


In [ ]:
# Medication cohorts: Insulin vs Oral meds vs None
MED_VALUES = {"Up", "Down", "Steady", "No"}

def is_med_col(series: pd.Series) -> bool:
    vals = set(series.dropna().astype(str).unique())
    return (len(vals) >= 2) and vals.issubset(MED_VALUES)

known_nonmed = {
    "race","gender","age","payer_code","medical_specialty","diag_1","diag_2","diag_3",
    "max_glu_serum","A1Cresult","change","diabetesMed","readmitted",
    "Primary_Diagnosis_Desc","readmit_label"
}

med_cols = [c for c in df.columns if (c not in known_nonmed and df[c].dtype == "object" and is_med_col(df[c]))]
oral_med_cols = [c for c in med_cols if c.lower() != "insulin"]

print("Oral medication columns detected (excluding insulin):", len(oral_med_cols))

def has_oral_med(row):
    return any(str(row[c]) in {"Up","Down","Steady"} for c in oral_med_cols)

df["on_insulin"] = df["insulin"].astype(str).isin({"Up","Down","Steady"}) if "insulin" in df.columns else False
df["on_oral_meds"] = df.apply(has_oral_med, axis=1) if oral_med_cols else False

def cohort(r):
    if r["on_insulin"]:
        return "Insulin"
    elif r["on_oral_meds"]:
        return "Oral meds only"
    else:
        return "No diabetes meds in oral cohort"

df["med_cohort"] = df.apply(cohort, axis=1)

cohort_rates = df.groupby("med_cohort")["readmit_30d"].mean().sort_values(ascending=False)
display(cohort_rates.to_frame("readmit_30d_rate"))

plt.figure(figsize=(7,4))
sns.barplot(x=cohort_rates.index, y=cohort_rates.values)
plt.title("30-Day Readmission Rate by Medication Cohort")
plt.xlabel("Cohort")
plt.ylabel("Rate")
plt.xticks(rotation=15)
show()


In [ ]:
# Medication change effect
if "change" in df.columns:
    change_rates = df.groupby("change")["readmit_30d"].mean().sort_values(ascending=False)
    display(change_rates.to_frame("readmit_30d_rate"))

    plt.figure(figsize=(5,4))
    sns.barplot(x=change_rates.index, y=change_rates.values)
    plt.title("30-Day Readmission Rate by Medication Change (change)")
    plt.xlabel("Medication change during stay")
    plt.ylabel("Rate")
    show()
else:
    print("No 'change' column found.")


In [ ]:
# time_in_hospital vs num_lab_procedures
plt.figure(figsize=(6,5))
sns.scatterplot(data=df, x="time_in_hospital", y="num_lab_procedures", alpha=0.3)
plt.title("time_in_hospital vs num_lab_procedures")
plt.xlabel("Time in hospital (days)")
plt.ylabel("Number of lab procedures")
show()


In [ ]:
# Correlation heatmap (numeric features)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in ["encounter_id", "patient_nbr"]:
    if c in num_cols:
        num_cols.remove(c)

corr = df[num_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=False)
plt.title("Correlation Heatmap (Numeric Features)")
show()

display(corr.round(2).iloc[:12,:12])


In [ ]:
# Boxplot: time_in_hospital by readmission (NO vs <30)
sub = df[df["readmitted"].isin(["NO", "<30"])].copy()

plt.figure(figsize=(6,4))
sns.boxplot(data=sub, x="readmitted", y="time_in_hospital", order=["NO","<30"])
plt.title("Length of Stay by Readmission Status (NO vs <30)")
plt.xlabel("Readmitted")
plt.ylabel("Time in hospital (days)")
show()


In [ ]:
# Discharge disposition analysis (top 10)
disp_col = "discharge_disposition_id_desc" if "discharge_disposition_id_desc" in df.columns else "discharge_disposition_id"

top_disp = df[disp_col].value_counts().head(10).index
tmp2 = df[df[disp_col].isin(top_disp)].copy()

disp_rates = tmp2.groupby(disp_col)["readmit_30d"].mean().sort_values(ascending=False)
display(disp_rates.to_frame("readmit_30d_rate"))

plt.figure(figsize=(12,4))
sns.barplot(x=disp_rates.index.astype(str), y=disp_rates.values)
plt.title("30-Day Readmission Rate by Discharge Disposition (Top 10)")
plt.xlabel("Discharge disposition")
plt.ylabel("Rate")
plt.xticks(rotation=30, ha="right")
show()


## Phase 4 — Feature Engineering: Vitality Complexity Index (VCI)
Implements the scoring logic exactly as specified and validates via readmission rates per risk band.

In [ ]:
# VCI scoring functions
def lace_L(days: int) -> int:
    if pd.isna(days): return 0
    if days < 1: return 0
    if 1 <= days <= 4: return 1
    if 5 <= days <= 13: return 4
    return 7  # >= 14

def lace_A(admission_type_id: int) -> int:
    if pd.isna(admission_type_id): return 0
    return 3 if int(admission_type_id) in {1, 7} else 0

def lace_C(num_dx: int) -> int:
    if pd.isna(num_dx): return 0
    if num_dx < 4: return 0
    if 4 <= num_dx <= 7: return 3
    return 5  # >= 8

def lace_E(num_er: int) -> int:
    if pd.isna(num_er): return 0
    if num_er == 0: return 0
    if 1 <= num_er <= 4: return 3
    return 5  # > 4

df["VCI_L"] = df["time_in_hospital"].apply(lace_L)
df["VCI_A"] = df["admission_type_id"].apply(lace_A)
df["VCI_C"] = df["number_diagnoses"].apply(lace_C)
df["VCI_E"] = df["number_emergency"].apply(lace_E)

df["VCI_Score"] = df[["VCI_L","VCI_A","VCI_C","VCI_E"]].sum(axis=1)

def vci_band(score: int) -> str:
    if score < 7: return "Low"
    if 7 <= score <= 10: return "Medium"
    return "High"

df["VCI_RiskBand"] = df["VCI_Score"].apply(vci_band)

display(df[["VCI_L","VCI_A","VCI_C","VCI_E","VCI_Score","VCI_RiskBand"]].head(10))


In [ ]:
# Validate VCI bands vs <30 readmission rate
band_rates = df.groupby("VCI_RiskBand")["readmit_30d"].mean().reindex(["Low","Medium","High"])
display(band_rates.to_frame("readmit_30d_rate"))

plt.figure(figsize=(5,4))
sns.barplot(x=band_rates.index, y=band_rates.values)
plt.title("VCI Risk Bands vs 30-Day Readmission Rate (<30)")
plt.xlabel("VCI Risk Band")
plt.ylabel("Rate")
show()


## Optional: Export cleaned/enriched data
Handy for your report writing and any later modeling.

In [ ]:
OUTPUT_CSV = "diabetic_data_clean_enriched.csv"
df.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV, "shape:", df.shape)
